In [1]:
import os
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1599231856298_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
### De forma temporal, importamos la metadata filtrada con solo 3 columnas
df_metadata = spark.read.csv("s3://info-proyecto-mineria-datos/products_metadata.csv", header = True).drop_duplicates(subset=['asin'])
#--df_metadata.show(4)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df_metadata_aux = spark.read.option("multiLine", True).option("mode", "PERMISSIVE").json("s3://info-proyecto-mineria-datos/All_Amazon_Meta.json.gz")
df_metad = spark.read.format("s3selectJson").option("compression", "gzip").schema(df_metadata_aux.schema).load("s3://info-proyecto-mineria-datos/All_Amazon_Meta.json.gz").limit(1000)
#--df_metad.show(6)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
### Cargamos la informacion de todos los reviews almacenados en S3
df_reviews = spark.read.csv("s3://amazon-reviews-pds/tsv/amazon_reviews_us_*.tsv.gz", sep= '\t', header = True)

### Imprimimos el esquema del conjunto de datos y mostramos las primeras 5 filas
#--df_reviews.printSchema()
#--df_reviews.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
### Numero de registros que tiene el conjunto de datos
#--print(f'La cantidad de reviews que tenemos es: {df_reviews.count()}')

In [5]:
### Al conjunto de datos de los reviews le agregamos la metadata de los productos
reviews = df_reviews.alias('reviews')
products = df_metadata.alias('products')

all_data = reviews.join(products, reviews.product_id == products.asin,how='left') # Could also use 'left_outer'
#--all_data.select('product_id','asin','star_rating','price','title').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#---all_data.count()

In [6]:
df_clean = all_data.na.drop(subset = ['asin','price','title','star_rating'])
#---df_clean.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#--f_clean.select('product_id','asin','star_rating','price','title').show()

In [ ]:
#--df_clean.agg(countDistinct(col("product_id")).alias("count")).show()

In [ ]:
#df_clean.select(countDistinct("star_rating")).show()

Validación categorias

In [ ]:
#--df_product_category = df_clean.groupBy('product_category').count()

In [ ]:
#--df_product_category.sort('count', ascending=False).show(61)

In [7]:
from pyspark.sql import SparkSession

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df_clean.createOrReplaceTempView("df_clean")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df_final = spark.sql("with category as (SELECT product_category, count(*) as n_reviews FROM df_clean  group by product_category), category_def as (select * from category where n_reviews >= 2) select t1.* from df_clean as t1 inner join category_def as t2 on t1.product_category = t2.product_category where t1.star_rating in ('1','2','3','4','5') ")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#df_final.show()

In [11]:
NH = df_final.groupBy('product_category').count() #Poblacion por cada pruduct category

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
n = 0.002

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
list_of_lat = NH.rdd.map(lambda r: (r.product_category,n)).collectAsMap()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
muestra = df_final.sampleBy("product_category", fractions=list_of_lat,seed=10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
muestra.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

73614

In [ ]:
muestra.groupBy('product_category').count().sort('count', ascending=False).show(41)

# Implementación de modelos

In [22]:
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit,CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml import Pipeline, PipelineModel


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Se eliminan del data frame del modelo las variables no objetivo

In [23]:
datos_modelo = muestra.drop('marketplace','review_id','product_parent','helpful_votes','total_votes',
                            'vine','verified_purchase','review_headline','review_body','review_date','asin','price','title')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
indexer_product_id = StringIndexer(inputCol="product_id", outputCol="item")
ratings_df = indexer_product_id.fit(muestra).transform(muestra)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
ratings_df.registerTempTable("temp2")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
query = """
SELECT 
  cast(customer_id as int) as customer_id, 
  cast(item as int) as item,
  product_id,
  cast(star_rating as int) as star_rating,
  product_title
FROM temp2 as t1
where cast(customer_id as int) is not null and
      cast(item as int) is not null and
      star_rating              is not null
"""
ratings_df = spark.sql(query)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
(df_training, df_test) = ratings_df.randomSplit([0.8, 0.2])
df_training.cache()
df_test.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[customer_id: int, item: int, product_id: string, star_rating: int, product_title: string]

In [29]:
#training.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----+----------+-----------+--------------------+
|customer_id| item|product_id|star_rating|       product_title|
+-----------+-----+----------+-----------+--------------------+
|      10981| 5501|9966541551|          4|Professional Ultr...|
|      34266|49048|B008HZW5AI|          5|10.80V,5200mAh,Li...|
|      58217|29218|B00HHKLB40|          4|Poetic Slimbook C...|
|     102629|26039|B000QSM0WC|          3|2in1 USB Cable Fo...|
|     121406|  827|B00KJ07SEM|          4|GE MWF SmartWater...|
|     129665|18336|B003ZWPS3O|          5|Alfa 2.4HGz 7dBi ...|
|     140594|59071|B00ISUDBCI|          5|Abacus24-7 micro ...|
|     173590| 3333|B00LLEODCK|          4|SanDisk 16GB CZ43...|
|     177441|  671|B00E3CHLLC|          3|Kaisi 16-Piece Pr...|
|     199482| 1125|B008ZT8QKO|          5|Hauppauge Persona...|
|     212448| 4921|B00H33SFJU|          5|EVGA 430W 80PLUS ...|
|     233613| 5179|B00EO6GSY8|          5|Fosmon [ETL Liste...|
|     238754|  455|B00DU2CHE2|          

# Modelo básico

In [34]:
als = ALS(userCol="customer_id", itemCol="item", ratingCol="star_rating",
          coldStartStrategy="drop")
model = als.fit(df_training)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
predictions = model.transform(df_test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="star_rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root-mean-square error = 4.67783169642135

In [40]:
def als_pipeline(df_train,df_test,s,rank):
    als = ALS(maxIter=5, rank =rank , regParam=s, userCol="customer_id", itemCol="item", ratingCol="star_rating",coldStartStrategy="drop")
    model = als.fit(df_train)
    predictions = model.transform(df_test)
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="star_rating",predictionCol="prediction")
    testset_rmse = evaluator.evaluate(predictions)
    return testset_rmse,s,rank

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
from time import time

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
downsamples = [0.01,0.5]# list of percentages to downsample training set
rango = [4,6] 
start_time = time()
rmses = [] 
for s in downsamples:
    for d in rango:
        test_rmse,s,rank = als_pipeline(df_training,df_test,s,d)
        elapsed_time = time() - start_time
        rmses.append([test_rmse,s,elapsed_time,rank])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
df_metrica = spark.createDataFrame(rmses,['rmse','regParam','tiempo','rank'])
df_metrica.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------+------------------+----+
|              rmse|regParam|            tiempo|rank|
+------------------+--------+------------------+----+
| 5.992908247592727|    0.01|30.462782382965088|   4|
| 5.156277633414702|    0.01| 61.37172293663025|   6|
| 5.291734005054755|     0.5| 91.81170129776001|   4|
|4.9708862859800345|     0.5| 122.7469711303711|   6|
+------------------+--------+------------------+----+

In [17]:
# transform asin and user alphanumeric string to index using spark StringIndexer function
asinIndexer = StringIndexer(inputCol="product_id", outputCol="item",handleInvalid='error') # create indexer for asins
userIndexer = StringIndexer(inputCol='customer_id',outputCol='userid',handleInvalid='error') # create indexer for user
asinIndexed = asinIndexer.fit(datos_modelo).transform(datos_modelo) # apply asin indexer
userIndexed = userIndexer.fit(asinIndexed).transform(asinIndexed) # apply user indexer
userIndexed.drop('product_id').drop('customer_id') # remove old columns with alphanumeric strings

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

KeyboardInterrupt: 

In [37]:
df_indexed_f = df_indexed.select(df_indexed.star_rating.cast("float"),
df_indexed.item.cast("double"),
df_indexed.userid.cast("double"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# 80-30 train-test split
(df_train, df_test) = df_indexed_f.randomSplit([0.8,0.2])
# cache them in memory across clusters since we access this data frequently 
df_train.cache() 
df_test.cache()

VBox()

In [ ]:
def als_pipeline(df_train,df_test,s,rank):
    rmses = [] 
    als = ALS(maxIter=5, rank =rank , regParam=s, userCol="userid", itemCol="item", ratingCol="star_rating",coldStartStrategy="drop")
    model = als.fit(df_train)
    predictions = model.transform(df_test)
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="star_rating",predictionCol="prediction")
    testset_rmse = evaluator.evaluate(predictions)
    return testset_rmse,s,rank

In [ ]:
downsamples = [0.01,0.8]# list of percentages to downsample training set
rango = [4,8] 
start_time = time()
rmses = [] 
for s in downsamples:
    for d in rango:
        test_rmse,s,rank = als_pipeline(df_train,df_test,s,d)
        elapsed_time = time() - start_time
        rmses.append([test_rmse,s,elapsed_time,rank])

In [ ]:
#-- datos_modelo.show(2)

In [ ]:
from pyspark.sql.types import *

In [ ]:
spark.sparkContext.getConf().get('spark.driver.memory')

# Baseline

In [ ]:
# mean = float(df_train.describe().toPandas()['rating'][1]) # mean

# ALS

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit,CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml import Pipeline, PipelineModel
from pyspark.mllib.evaluation import RegressionMetrics

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
>>> df1 = sc.parallelize([['1',2,3], [2,3,1],['1',2,2], [2,3,2],['1.',3,3], [2,3,4],['1',2,3], [2,3,4],['1',2,3], [2,3,4]]).toDF(("star_rating", "userids", "product1"))
>>> df1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------+--------+
|star_rating|userids|product1|
+-----------+-------+--------+
|          1|      2|       3|
|          2|      3|       1|
|          1|      2|       2|
|          2|      3|       2|
|         1.|      3|       3|
|          2|      3|       4|
|          1|      2|       3|
|          2|      3|       4|
|          1|      2|       3|
|          2|      3|       4|
+-----------+-------+--------+

In [5]:
# transform asin and user alphanumeric string to index using spark StringIndexer function
asinIndexer = StringIndexer(inputCol="product1", outputCol="item",handleInvalid='error') # create indexer for asins
userIndexer = StringIndexer(inputCol='userids',outputCol='userid',handleInvalid='error') # create indexer for user
asinIndexed = asinIndexer.fit(df1).transform(df1) # apply asin indexer
userIndexed = userIndexer.fit(asinIndexed).transform(asinIndexed) # apply user indexer
df_indexed = userIndexed.drop('product1').drop('userids') # remove old columns with alphanumeric strings

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df_indexed.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----+------+
|star_rating|item|userid|
+-----------+----+------+
|          1| 0.0|   1.0|
|          2| 3.0|   0.0|
|          1| 2.0|   1.0|
|          2| 2.0|   0.0|
|         1.| 0.0|   0.0|
|          2| 1.0|   0.0|
|          1| 0.0|   1.0|
|          2| 1.0|   0.0|
|          1| 0.0|   1.0|
|          2| 1.0|   0.0|
+-----------+----+------+

In [7]:
df_indexed_f = df_indexed.select(df1.star_rating.cast("float"),
df_indexed.userid.cast("double"),
df_indexed.item.cast("double"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# 80-30 train-test split
(df_train, df_test) = df_indexed_f.randomSplit([0.8,0.2])
# cache them in memory across clusters since we access this data frequently 
df_train.cache() 
df_test.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[star_rating: float, userid: double, item: double]

In [20]:
als = ALS(maxIter=5, rank =2 , regParam=0.01, userCol="userid", itemCol="item", ratingCol="star_rating",
          coldStartStrategy="drop")
model = als.fit(df_train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
df_test.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------+----+
|star_rating|userid|item|
+-----------+------+----+
|        1.0|   1.0| 0.0|
|        1.0|   0.0| 0.0|
|        1.0|   1.0| 0.0|
|        1.0|   1.0| 0.0|
+-----------+------+----+

In [23]:
predictions = model.transform(df_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
predictions = model.transform(df_test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="star_rating",
                                predictionCol="prediction")
#rmse = evaluator.evaluate(predictions)
#print("Root-mean-square error = " + str(rmse))
predictions.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------+----+----------+
|star_rating|userid|item|prediction|
+-----------+------+----+----------+
+-----------+------+----+----------+

In [10]:
from pyspark.sql import Row
from pyspark.sql import SparkSession

from pyspark.ml.feature import StringIndexer
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit,CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml import Pipeline, PipelineModel

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
def als_pipeline(df_train,df_test,s,rank):
    als = ALS(maxIter=5, rank =rank , regParam=s, userCol="userid", itemCol="item", ratingCol="star_rating",coldStartStrategy="drop")
    model = als.fit(df_train)
    predictions = model.transform(df_test)
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="star_rating",predictionCol="prediction")
    testset_rmse = evaluator.evaluate(predictions)
    return testset_rmse,s,rank

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
downsamples = [0.01,0.8]# list of percentages to downsample training set
rango = [4,8] 
start_time = time()
rmses = [] 
for s in downsamples:
    for d in rango:
        test_rmse,s,rank = als_pipeline(df_train,df_test,s,d)
        elapsed_time = time() - start_time
        rmses.append([test_rmse,s,elapsed_time,rank])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'requirement failed: Nothing has been added to this summarizer.'
Traceback (most recent call last):
  File "<stdin>", line 6, in als_pipeline
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/ml/evaluation.py", line 71, in evaluate
    return self._evaluate(dataset)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/ml/evaluation.py", line 101, in _evaluate
    return self._java_obj.evaluate(dataset._jdf)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 79, in deco
    raise IllegalArgumentException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.IllegalArgumentException: 'requirement failed: Nothing has been added to this summarizer.'



In [13]:
from time import time

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df_metrica = spark.createDataFrame(rmses,['rmse','regParam','tiempo','rank'])
df_metrica.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

can not infer schema from empty dataset
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 748, in createDataFrame
    rdd, schema = self._createFromLocal(map(prepare, data), schema)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 416, in _createFromLocal
    struct = self._inferSchemaFromList(data, names=schema)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 343, in _inferSchemaFromList
    raise ValueError("can not infer schema from empty dataset")
ValueError: can not infer schema from empty dataset

